In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Function to load images and labels from a directory
def load_images_from_directory(directory):
    images = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                try:
                    image = Image.open(image_path).convert('RGB')
                    image = image.resize((64, 64))
                    image = np.array(image) / 255.0
                    images.append(image)
                    labels.append(class_name)
                except Exception as e:
                    print(f"Error loading image {image_path}: {str(e)}")
    return np.array(images), np.array(labels)

In [ ]:
# Define dataset directory (Update this before running)
data_directory = "dataset/"
train_images, train_labels = load_images_from_directory(data_directory)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, 
                train_labels, test_size=0.2, random_state=42)

In [ ]:
# Get class labels
class_labels = label_encoder.classes_
print("Number of classes:", len(class_labels))
print("Class labels:", class_labels)


In [ ]:
# Display some sample images
plt.figure(figsize=(15, 15))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    plt.xlabel(class_labels[y_train[i]])
plt.show()

In [ ]:
# Define the CNN model
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=[64, 64, 3]),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=len(class_labels), activation='softmax')
])

In [ ]:
# Compile the model
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = cnn.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(history.history['accuracy']) + 1), history.history['accuracy'], label='Training Accuracy')
plt.plot(range(1, len(history.history['val_accuracy']) + 1), history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(history.history['loss']) + 1), history.history['loss'], label='Training Loss')
plt.plot(range(1, len(history.history['val_loss']) + 1), history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the model (Update this path before running)
model_path = "saved_model/cnn_model.h5"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
cnn.save(model_path)

In [ ]:
# Evaluate the model
y_val_pred = np.argmax(cnn.predict(X_val), axis=1)
print("Validation Set Classification Report:")
print(classification_report(y_val, y_val_pred, target_names=class_labels))

y_train_pred = np.argmax(cnn.predict(X_train), axis=1)
print("Training Set Classification Report:")
print(classification_report(y_train, y_train_pred, target_names=class_labels))

In [ ]:
# Function to predict a single image
def predict_image(image_path):
    test_image = Image.open(image_path).convert('RGB')
    test_image_display = test_image.copy()
    test_image = test_image.resize((64, 64))
    test_image_array = np.array(test_image) / 255.0
    test_image_array = np.expand_dims(test_image_array, axis=0)
    
    prediction = cnn.predict(test_image_array)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class] * 100
    
    plt.figure(figsize=(8, 8))
    plt.imshow(test_image_display)
    plt.axis('off')
    plt.title(f"Predicted: {class_labels[predicted_class]}\nConfidence: {confidence:.2f}%")
    plt.show()
    
    print(f"The image is classified as '{class_labels[predicted_class]}' with {confidence:.2f}% confidence.")

In [ ]:
# Example usage (Update this path before running)
test_image_path = "test_images/sample.jpg"
predict_image(test_image_path)